In [1]:
import pandas as pd
import ta

In [2]:
df = pd.read_csv('../BTC_ETH_1week_1m.csv')

In [3]:
symbols = ['ETH']

for symbol in symbols:
    df[f'{symbol}_RSI'] = ta.momentum.rsi(close=df[f'{symbol}-USD_Close'], window=20)
    df[f'{symbol}_ATR'] = ta.volatility.average_true_range(high=df[f'{symbol}-USD_High'], low=df[f'{symbol}-USD_Low'], close=df[f'{symbol}-USD_Close'], window=14)

df.dropna(inplace=True)

In [4]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, symbols):
        self.short_unit = None
        self.short_amount = 0
        self.short_price = 0
        self.long_price = 0

        self.sl = 0
        self.slm = 5

        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.symbols = symbols
        
        self.buys = []
        self.sells = []
        
        self.wins = 0
        self.losses = 0
        self.ties = 0
        
        self.trading_fee_multiplier = 0.99945      # VIP level 0, paying fees with BNB 0.99975
        self.number_of_buys = len(self.buys)

    def buy_long(self, symbol, long_price, atr, time):
        self.balance_amount = (self.balance_amount / long_price) * self.trading_fee_multiplier
        self.long_price = long_price
        self.balance_unit = symbol
        self.buys.append([symbol, time, long_price, 'long'])
        self.number_of_buys = len(self.buys)
        self.sl = long_price - (self.slm*atr)
        print('filled long', long_price, time, '\n\n')

    def buy_short(self, symbol, short_price, atr, time):
        self.balance_amount = self.balance_amount * self.trading_fee_multiplier
        self.short_amount = (self.balance_amount / short_price) 
        self.short_price = short_price
        self.short_unit = symbol
        self.buys.append([symbol, time, short_price, 'short'])
        self.number_of_buys = len(self.buys)
        self.sl = short_price + (env.slm*atr)
        print('filled short', short_price, time, '\n\n')

    def sell_long(self, sell_price, time):
        self.balance_amount = (self.balance_amount * sell_price) * self.trading_fee_multiplier
        self.sells.append([symbol, time, sell_price])
        self.balance_unit = 'BUSD'
    
    def sell_short(self, sell_price, time):
        self.balance_amount = (self.balance_amount + (self.short_amount * ((self.short_price*self.trading_fee_multiplier) - sell_price)))
        self.sells.append([symbol, time, sell_price])
        self.short_unit = None
        self.short_amount = 0
    


In [5]:
"""
BUY:
    Long: 
        1. Check if symbol crosses under 30 on RSI -> Buy

    Short:
        1. Check if symbol crosses over 70 on RSI

"""

'\nBUY:\n    Long: \n        1. Check if symbol crosses under 30 on RSI -> Buy\n\n    Short:\n        1. Check if symbol crosses over 70 on RSI\n\n'

In [6]:
env = TradingEnv(balance_amount=100, balance_unit='BUSD', symbols=symbols)
for i in range(len(df)):    
    print(env.balance_amount, env.balance_unit, env.number_of_buys)
    if env.balance_unit == 'BUSD' and env.short_unit == None:  # Want to buy
        for symbol in symbols:

            ######## LONG           (lower + ((upper-lower)*multiplier))
            if df[f'{symbol}_RSI'].iloc[i] <= 30:
                env.buy_long(symbol, long_price=df[f'{symbol}-USD_Close'].iloc[i], atr=df[f'{symbol}_ATR'].iloc[i], time=df['OpenTime'].iloc[i])
                break
    

            ######## SHORT
            # if df[f'{symbol}_RSI'].iloc[i] >= 70:
            #     env.buy_short(symbol, short_price=df[f'{symbol}-USD_Close'].iloc[i], atr=df[f'{symbol}_ATR'].iloc[i], time=df['OpenTime'].iloc[i])
            #     break



    ####################################################################################################################

    ############## Want to sell long position
    elif env.balance_unit != 'BUSD':

        # Sell long for profit
        if df[f'{symbol}_RSI'].iloc[i] >= 70:
            print('sell long', df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.wins += 1
            env.sell_long(sell_price=df[f'{env.balance_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            continue

        elif df[f'{env.balance_unit}-USD_Close'].iloc[i] <= env.sl:
            print('Stop loss hit: sell long', df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.losses += 1
            env.sell_long(sell_price=df[f'{env.balance_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            continue

    ############## Want to sell short position
    
    # else:
    #     print(env.short_amount)
    #     # Sell short for profit
    #     if env.cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_Close'].iloc[i] >= (df[f'{symbol}_Lower_Band'].iloc[i] + ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)):
    #         print('sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
    #         env.wins += 1
    #         env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
    #         env.reset_crosses(symbols)
    #         continue
        
    #     # Sell short at stop loss
    #     elif df[f'{env.short_unit}-USD_Close'].iloc[i] >= env.sl:
    #         print('Stop loss hit: sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
    #         env.losses += 1
    #         env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
    #         env.reset_crosses(symbols)
    #         continue
        
    #     # Sell short at break-even  
    #     if env.sma_cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_High'].iloc[i] >= env.short_price:
    #         print('Break even hit: sell short', env.short_price, df['OpenTime'].iloc[i], '\n\n')
    #         env.ties += 1
    #         env.sell_short(sell_price=env.short_price, time=df['OpenTime'].iloc[i])
    #         env.reset_crosses(symbols)
    #         continue

    #     # Check if SMA is crossed down
    #     if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_SMA'].iloc[i]:
    #         print('SMA short crossed')
    #         env.sma_cross_down[env.short_unit] = True

    #     # Check if lower band is crossed
    #     if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_Lower_Band'].iloc[i]:
    #         print('short crossed')
    #         env.cross_down[env.short_unit] = True
    




if env.balance_unit != 'BUSD':
    env.sell_long(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
if env.short_unit != None:
    env.sell_short(df[f'{env.short_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
print(env.balance_amount, env.balance_unit, env.number_of_buys, env.wins, env.ties, env.losses)

100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0

In [7]:

data = pd.DataFrame([(i[1] for i in env.buys), (j[2] for j in env.buys), (a[3] for a in env.buys), (k[1] for k in env.sells), (l[2] for l in env.sells)])
data

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,2021-01-07 21:30:00,2021-01-10 15:15:00,2021-01-15 11:00:00,2021-01-20 06:00:00,2021-01-21 17:15:00,2021-01-25 17:00:00,2021-02-06 21:00:00,2021-02-07 10:15:00,2021-02-14 20:30:00,2021-02-14 21:30:00,...,2021-12-09 10:15:00,2021-12-10 19:00:00,2021-12-12 20:15:00,2021-12-13 10:45:00,2021-12-17 02:30:00,2021-12-20 04:45:00,2021-12-22 07:45:00,2021-12-27 20:30:00,2021-12-28 12:15:00,2021-12-29 18:30:00
1,1071.99,1199.83,1128.71,1277.63,1106.33,1315.48,1636.76,1538.55,1757.36,1715.89,...,4164.05,3859.12,3995.75,3806.29,3869.42,3767.58,3954.04,3980.01,3803.22,3618.28
2,long,long,long,long,long,long,long,long,long,long,...,long,long,long,long,long,long,long,long,long,long
3,2021-01-09 16:00:00,2021-01-10 22:30:00,2021-01-18 20:15:00,2021-01-21 11:45:00,2021-01-23 22:45:00,2021-02-01 17:00:00,2021-02-07 10:00:00,2021-02-08 07:45:00,2021-02-14 21:15:00,2021-02-17 07:15:00,...,2021-12-10 11:00:00,2021-12-12 13:00:00,2021-12-13 10:30:00,2021-12-15 15:00:00,2021-12-17 09:30:00,2021-12-20 16:00:00,2021-12-23 12:45:00,2021-12-28 04:45:00,2021-12-29 18:15:00,2021-12-31 19:14:59.999
4,1285.7,1043.81,1310.45,1172.45,1294.62,1360.7,1562.0,1733.28,1686.73,1845.9,...,3987.73,4135.23,3848.79,4034.83,3706.02,3959.08,4042.0,3884.55,3636.71,3691.5
